<a href="https://colab.research.google.com/github/hyomin14/python_ML_practice/blob/main/09_Recommendations/Ch9_08_Surprise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 파이썬 추천 시스템 패키지 - Surprise

### Surprise를 이용한 추천 시스템 구축

In [4]:
!pip install scikit-surprise

     |████████████████████████████████| 11.8 MB 24.3 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1623324 sha256=69057e45a162b678aa303c0b18323e07277e4130a33b6a9b17b0d8a6680355b4
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [5]:
from surprise import SVD, Dataset, accuracy
from surprise.model_selection import train_test_split

In [6]:
data = Dataset.load_builtin('ml-100k')  #과거 버전의 데이터 파일로 분리 문자가 탭(\t) 문자임
#수행시마다 동일하게 데이터 분할하기 위해 random_state값 부여
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [11]:
type(trainset)

surprise.trainset.Trainset

In [7]:
algo = SVD()
algo.fit(trainset)

In [8]:
#test(): 사용자-아이템 평점 데이터 세트 전체에 대해서 추천 예측
#predict(): 개별 사용자와 영화에 대한 추천 평점 반환
predictions = algo.test(testset)
print('prediction type:', type(predictions), ', size:', len(predictions))
print('predictions 결과의 최초 5개 추출')
predictions[:5]

prediction type: <class 'list'> , size: 25000
predictions 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.638652144331622, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.8411408611067723, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=3.897371767289689, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.5401836913661224, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.447863194068727, details={'was_impossible': False})]

uid: 개별 사용자 아이디, iid: 영화나 아이템 아이디, r_ui: 실제 평점, est: 추천 예측 평점

details: 내부 처리 시 추천 예측을 할 수 없는 경우 로그용으로 데이터를 남기는데 사용 - True이면 예측값 생성할 수 없음

In [12]:
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.638652144331622),
 ('882', '291', 3.8411408611067723),
 ('535', '507', 3.897371767289689)]

In [13]:
#사용자 아이디, 아이템 아이디는 문자열로 입력해야함
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.28   {'was_impossible': False}


In [14]:
accuracy.rmse(predictions)

RMSE: 0.9464


0.9464128894921023

#### OS 파일 데이터를 Surprise 데이터 세트로 로딩

In [15]:
import pandas as pd

ratings = pd.read_csv('ratings.csv')
ratings.to_csv('ratings_noh.csv', index=False, header=False)

In [16]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data = Dataset.load_from_file('ratings_noh.csv', reader=reader)

In [17]:
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

#### pandas DataFrame에서 Surprise 데이터 세트로 로딩

In [18]:
ratings = pd.read_csv('ratings.csv')
reader = Reader(rating_scale=(0.5, 5.0))

data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

### 교차 검증과 하이퍼 파라미터 튜닝

In [19]:
from surprise.model_selection import cross_validate

#pandas DataFrame에서 Surprise 데이터 세트로 데이터 로딩
ratings = pd.read_csv('ratings.csv')
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8771  0.8674  0.8721  0.8728  0.8788  0.8737  0.0040  
MAE (testset)     0.6771  0.6647  0.6710  0.6735  0.6732  0.6719  0.0041  
Fit time          9.40    7.52    6.43    6.13    7.03    7.30    1.15    
Test time         0.31    0.36    0.21    0.51    0.29    0.34    0.10    


{'fit_time': (9.397850513458252,
  7.518667459487915,
  6.434463262557983,
  6.130863428115845,
  7.031172513961792),
 'test_mae': array([0.67714348, 0.66465789, 0.67096788, 0.67345602, 0.67320181]),
 'test_rmse': array([0.87706137, 0.86744847, 0.87210519, 0.8728253 , 0.87884626]),
 'test_time': (0.305645227432251,
  0.36110973358154297,
  0.21231603622436523,
  0.5113053321838379,
  0.28947949409484863)}

In [20]:
from surprise.model_selection import GridSearchCV

#최적화할 파라미터를 딕셔너리 형태로 지정
param_grid = {'n_epochs':[20, 40, 60], 'n_factors':[50, 100, 200]}

#cv를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse로 수행하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3, joblib_verbose=2)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.4s remaining:    0.0s


0.8771505989593898
{'n_epochs': 20, 'n_factors': 50}


[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  5.1min finished


### Surprise를 이용한 개인화 영화 추천 시스템 구축

In [22]:
#데이터 세트 전체를 학습 데이터로 사용하려면 DatasetAutoFolds 클래스 이용
from surprise.dataset import DatasetAutoFolds

ratings = pd.read_csv('ratings.csv')
ratings.to_csv('ratings_noh.csv', index=False, header=False)
reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
#DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성
data_folds = DatasetAutoFolds(ratings_file='ratings_noh.csv', reader=reader)

#전체 데이터를 학습 데이터로 생성
trainset = data_folds.build_full_trainset()

In [23]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [26]:
#영화에 대한 상세 속성 정보 DataFrame 로딩
movies = pd.read_csv('movies.csv')

#userId=9의 movieId 데이터를 추출해 movieId=42인 데이터가 있는지 확인
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count()==0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')
print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [27]:
movieIds

1073      41
1074     187
1075     223
1076     371
1077     627
1078     922
1079     923
1080    1037
1081    1095
1082    1198
1083    1270
1084    1674
1085    1987
1086    2011
1087    2012
1088    2023
1089    2300
1090    2877
1091    2901
1092    3173
1093    3328
1094    3735
1095    4131
1096    4558
1097    4993
1098    5218
1099    5378
1100    5445
1101    5447
1102    5451
1103    5481
1104    5507
1105    5841
1106    5843
1107    5872
1108    5890
1109    5891
1110    5893
1111    5902
1112    5952
1113    5956
1114    5962
1115    5965
1116    5988
1117    6001
1118    6044
Name: movieId, dtype: int64

In [28]:
uid = str(9)
iid = str(42)
pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [30]:
#평점 매기지 않은 전체 영화 추출
def get_unseen_surprise(ratings, movies, userId):
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId']==userId]['movieId'].tolist()

    #모든 영화의 movieId를 리스트로 생성
    total_movies = movies['movieId'].tolist()

    #모든 영화의 movieId 중 이미 평점 매긴 영화의 movieId 제외한 후 리스트로 생성
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화 수:', len(seen_movies), ', 추천 대상 영화 수:', len(unseen_movies), 
          ', 전체 영화 수:', len(total_movies))
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화 수: 46 , 추천 대상 영화 수: 9696 , 전체 영화 수: 9742


In [35]:
#인자로 학습 완료된 추천 알고리즘 객체, 추천 대상 사용자 아이디, 추천 대상 영화의 리스트, 추천 상위 N개 받음
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    #알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

    #predictions list 객체는 surprise의 Predictions 객체를 원소를 가지고 있음
    #[Predictions(uid='9', iid='1', est=3.69), ...]

    #이를 est 값으로 정렬하기 위해서는 아래의 sortkey_est 함수 정의
    #sortkey_est 함수는 list 객체의 sort()함수의 키 값으로 사용되어 정렬 수행
    def sortkey_est(pred):
        return pred.est
    
    #sortkey_est() 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions = predictions[:top_n]

    # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [ int(pred.iid) for pred in top_predictions]
    top_movie_rating = [ pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)

print('##### Top-10 추천 영화 리스트 #####')
for top_movie in top_movie_preds:
    print(top_movie[1], ':', top_movie[2])
    print()

##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.306302135700814

Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387

Pulp Fiction (1994) : 4.278152632122759

Silence of the Lambs, The (1991) : 4.226073566460876

Godfather, The (1972) : 4.1918097904381995

Streetcar Named Desire, A (1951) : 4.154746591122658

Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504

Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436

Goodfellas (1990) : 4.083464936588478

Glory (1989) : 4.07887165526957



In [37]:
predictions = [algo.predict(str(9), str(movieId)) for movieId in unseen_movies]
predictions[:10]

[Prediction(uid='9', iid='1', r_ui=None, est=3.6398015009715396, details={'was_impossible': False}),
 Prediction(uid='9', iid='2', r_ui=None, est=3.0708535569743463, details={'was_impossible': False}),
 Prediction(uid='9', iid='3', r_ui=None, est=2.9912517694217033, details={'was_impossible': False}),
 Prediction(uid='9', iid='4', r_ui=None, est=2.7584914835669805, details={'was_impossible': False}),
 Prediction(uid='9', iid='5', r_ui=None, est=2.653809583804632, details={'was_impossible': False}),
 Prediction(uid='9', iid='6', r_ui=None, est=3.7627640786692917, details={'was_impossible': False}),
 Prediction(uid='9', iid='7', r_ui=None, est=2.883779132627597, details={'was_impossible': False}),
 Prediction(uid='9', iid='8', r_ui=None, est=2.9865774263273672, details={'was_impossible': False}),
 Prediction(uid='9', iid='9', r_ui=None, est=2.7889711267915422, details={'was_impossible': False}),
 Prediction(uid='9', iid='10', r_ui=None, est=3.3040663633820104, details={'was_impossible': 